In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, metrics, svm,model_selection
from IPython.display import Image
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import math
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline  
import nltk

In [4]:
review_data=pd.read_csv("/home/dexterslaj/FireTablet1.csv")
review_data.head(n=3)

,ProductName,Reviews,Rating
0,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",The sales people are really nice and knowledge...,5.0
1,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",Great product and service and the sales man r ...,5.0
2,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",Great product and service will refer to a frie...,5.0


In [5]:
review_data.drop('ProductName',1,inplace=True)
review_data.head(n=3)

,Reviews,Rating
0,The sales people are really nice and knowledge...,5.0
1,Great product and service and the sales man r ...,5.0
2,Great product and service will refer to a frie...,5.0


In [6]:
x=[]
for i in range(10961):
    x.append(str(i))

    
nltk.download('punkt')    

[nltk_data] Downloading package punkt to /home/dexterslaj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
  
vocabulary={'null'}
print(type(vocabulary))
for i in range(10961):
    temp_review=review_data.iloc[[i]]
    temp_message=str(temp_review[['Reviews']])
   # print(type(temp_message))
    tempwords=word_tokenize(temp_message)
    for w in tempwords:
        if  w  in x:
            tempwords.remove(w)
   # print (tempwords)  
    for w in tempwords:
        vocabulary |= set(tempwords)

<class 'set'>


In [8]:
print(len(vocabulary))

5477


In [9]:
# adding a word  "agyat" (hindi) which is "unknown" in english hence this "agyat" will serve as unknown category..
vocabulary.add('agyat')

In [10]:
print(len(vocabulary))
#import nltk
#nltk.download('stopwords')

5478


In [11]:
# stopword removal
stopwordlist=list(stopwords.words('english'))
print(stopwordlist)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
stopwordslist1=[]
for i in range(116):
    stopwordslist1.append(stopwordlist[i])

In [15]:
print(stopwordslist1)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such']


In [17]:
stopvocab=[]
for w in vocabulary:
   # print(w)
    if w in stopwordslist1:
        continue
    else:    
        stopvocab.append(w)

In [18]:
print(len(stopvocab))

5376


In [19]:
i=0
for w in stopvocab:
    stopvocab[i]=w.lower()
    i=i+1

In [20]:
#removing digits and special charachters
j=0
for w in stopvocab:
    i=0
    temp=list(w)
    for c in temp:
        if c>='a' and c<='z':
            continue
        else:
            temp[i]=''
        i=i+1    
    stopvocab[j]="".join(temp)
    temp=[]
    j=j+1

In [21]:
print(len(stopvocab))

5376


In [22]:
# removing non english words
# SAD NEWS THAT SUPPORT FOR PYENCHANT HAS BEEN ENDED FOR ANACONDA ENVIROMENT SO NOW WE HAVE TO USE AN ALTERNATIVE IN ORDER TO REMOVE NON-ENGLISH WORDS
#import pyenchant
#d=enchant.Dict("en-us")

#enstopvocab=[]
#for w in stopvocab:
    #enstopvocab.append(w)
    
#for w in stopvocab:
    
    #if w!="" and d.check(w)==False:
        #enstopvocab.remove(w)
        

In [23]:
#nltk.download('wordnet')

In [24]:
from nltk.corpus import wordnet
enstopvocab=[]
for w in stopvocab:
    enstopvocab.append(w)
for w in stopvocab:
    if w!="" and  not wordnet.synsets(w):
        enstopvocab.remove(w)

In [25]:
print(len(enstopvocab))

3587


In [26]:
enstopvocab=list(set(enstopvocab))
print(len(enstopvocab))

2645


In [27]:
#oops our added word was removed since it was not an english word hence readding.
enstopvocab.append('agyat')
#thus the last position or index is for 'agyat' or unknown

In [28]:
# print(enstopvocab[0]) now here is a problem that first word in enstopvocab is null so we have to remove it
enstopvocab.remove("")

In [29]:
print(len(enstopvocab))

2645


In [30]:
# making vectors 1) one hot encoding 2) count vector
# using vovab enstopvocab
# using one-hot encodeing
def messagetoonehotvector(x):    
    messagevector=np.zeros(3620,dtype=int)
    temp1=word_tokenize(x)
    temp2=[]
    #for x in temp1:
       # x=lemmatizer.lemmatize(x)
        #temp2.append(x)
    for x in temp1:
        if x in stopwordslist:
            temp1.remove(x)
            
    for x in temp1:
        if x in enstopcovab:
            i=enstopvoab.index(x)
            messagevector[i]=1
            
            
    return list(messagevector)

In [31]:
countvect_esv=feature_extraction.text.CountVectorizer(vocabulary=enstopvocab)
tfidfvect=feature_extraction.text.TfidfVectorizer()
tfidfvect_esv=feature_extraction.text.TfidfVectorizer(vocabulary=enstopvocab)

In [32]:
X_cv=countvect_esv.transform(review_data['Reviews'])

In [33]:
np.shape(X_cv)

(10962, 2645)

In [34]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = model_selection.train_test_split(X_cv, review_data['Rating'], test_size=0.33, random_state=42)

In [35]:
#there is nothing in file so no NAN entry
x=np.array(y_train_cv)
count1=0
count2=0
count3=0
count4=0
count5=0
for p in x:
    if p==1:
        count1+=1
    elif p==2:
        count2+=1
    elif p==3:
        count3+=1
    elif p==4:
        count4+=1
    elif p==5:
        count5+=1
    
print(count1)
print(count2)
print(count3)
print(count4)
print(count5)

111
101
458
2348
4326


In [36]:
#nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn

In [37]:
#myfile1=open("SentiWordNet_3.0.0.txt",'r')
#myfile2=open("polaritydatabase.csv",'w')

In [38]:
#writer2=csv.writer(myfile2)

In [39]:
#csv_header=['POS','ID','POSSCORE','NEGSCORE','SYNSETTERMS','GLOSS']

In [40]:
#writer2.writerow(csv_header)

In [41]:
#writer2.writerow(['popo','2222'])
ssyn1=open('sentiwordnetsynsets1.txt','w')# this file will contain all the synsets for each word from enstopvocab

In [42]:
print(enstopvocab[0])
#enstopvocab.remove('')
print(len(enstopvocab))

continues
2645


In [43]:
enstopvocab=list(set(enstopvocab))
print(len(enstopvocab))

2645


In [44]:
polscore2=open('polscores2.txt','w')

In [45]:
tempsyn=[]
for w in enstopvocab:
    tempsyn=swn.senti_synsets(w)
    #polscore2.write("word is "+ str(w)+" ")
    #if len(list(tempsyn))==0:
        #ssyn1.write((str(x)+"  "))
        #polscore2.write((str(0.05)+" "+str(0.04)+","))
    for x in tempsyn:        
        ssyn1.write((str(x)+"  "))
        polscore2.write((str(x.pos_score())+" "+str(x.neg_score())+","))
    ssyn1.write("\n")
    polscore2.write("\n")                    
    tempsyn=[]
    

In [46]:
ssyn1.close()# finally we have a record of all synsets for ech word in enstopvocab
polscore2.close()


In [47]:
# A BIG NOTE :----- WE ARE USING SENTI WORD NET INTERFACE OF NLTK THAT COMPLETELY RELIES UPON SENTI WORD NET DATABASE.
# A QUICK OVERVIEW
# CURRENTLY WE ARE FINDING THE POLARITY OF EACH WORD
# NOW AS WE KNPOW THAT EACH WORD HAS SEVERAL SYNSETS DEPENDING ON PART OF SPEECH , CONTEXTUAL USE ETC.
# SINCE THIS IS OUR FIRST APPROACH TOWARDS POLARITY CLACULATION SO WE ARE JUST TAKING AREITHMATIC AVERAGE OVER ALL THE SYNSETS OF A WORD REGARDLESS OF ITS POS  OR CONTEXTUAL USE
# WE MAY GO FOR FURTHER ADVANCEMENT OF OUR ALGO..
# the approach we are using is not weaker in any sense . TEXTBLOB a famous sentiment analysis library uses this approach
avgposscore=[]# to hold avg pos polarity score of each word
avgnegscore=[] # to hold avg neg polarity score of each word

In [48]:
temppos=0.0
tempneg=0.0
count=0
polscore2=open('polscores2.txt','r')
temp=""
templist=[]
ptemp=[]
tempsyn=[]
i=1
for temp in polscore2:
    #for line in enstopvocab:
    templist=temp.split(',')
    #if i<=10:
    #print(str(i)+"):--")
    #print(temp)
    #print("\n")
    #i=i+1    
    #tempsyn=swn.senti_synsets(x)
    #count=len(list(tempsyn))# toatl no. of synsets returned
    for x in templist:
        if len(x)!=0:
            ptemp=x.split()
            #if i<=10:        
            #print(ptemp)
            #print("\n")
            if len(ptemp)!=0:
                temppos=temppos+float(ptemp[0])
                tempneg=tempneg+float(ptemp[1])
            ptemp=[]
        
    #print (count)
    #print(type(tempsyn))
    #print(tempsyn)
   # for w in tempsyn:
        
       # print(w.pos_score())
        #temppos=temppos+w.pos_score()# total pos score
        #tempneg=tempneg+w.neg_score()# total neg score
    #print(tempos)
    #print(tempneg)
    #tempos=temppos/count
    #tempneg=tempneg/count
    avgposscore.append(temppos)
    avgnegscore.append(tempneg)
    temppos=0.0
    tempneg=0.0
    count=0
    tempsyn=[]
    templist=[]
    i=i+1
    

    

In [49]:
# now recording these avg polarity scores
#polscore1=open('polscores1.txt','w')
#polscore1.write("pos score                neg score")
#for x in range(2641):
    #polscore1.write((str(avgposscore[x])+"       "+str(avgnegscore[x])))
    #polscore1.write("\n")

#print(avgposscore)
count=0
for x in range(2642):
    if enstopvocab[x]!='agyat':
        temp=swn.senti_synsets(enstopvocab[x])
        count=len(list(temp))
        #print(count)
        if count==0:
            print(enstopvocab[x])
        avgposscore[x]=avgposscore[x]/count
        avgnegscore[x]=avgnegscore[x]/count
    
 

In [50]:
print(enstopvocab[2502])
print(avgposscore[2502])
print(avgnegscore[2502])
#print(avgposscore) 

recognizing
0.16666666666666666
0.0


In [51]:
# now multiplying polarity with X_cv
print(type(X_cv))
fpol=np.zeros(shape=2642,dtype=float)
np.shape(fpol)
for i in range(2642):
    fpol[i]=avgposscore[i]+((-1)*avgnegscore[i])
    

<class 'scipy.sparse.csr.csr_matrix'>


In [52]:
print(fpol)

[0.     0.     0.0625 ... 0.     0.     0.    ]


In [53]:
X_cv1=X_cv.toarray()
#print(X_cv1)
#for i in range(10962):
    #for j in range(2642):
        #X_cv1[i][j]=(X_cv1[i][j]*fpol[j])
        

In [54]:
X_train_cv, X_test_cv, y_train_cv, y_test_cv = model_selection.train_test_split(X_cv1, review_data['Rating'], test_size=0.33, random_state=42)


In [66]:
model = Sequential()
model.add(Dense(1000, input_dim=2645, activation='relu'))
#model.add(Dense(4000, activation='softmax'))
model.add(Dense(500,activation='relu'))
#model.add(Dense(150, activation='softmax'))
model.add(Dense(5, activation='softmax'))

In [67]:
print(np.shape(X_train_cv))

(7344, 2645)


In [68]:
#now making a matrix of size 10962*5 which will serve as a y (train label vectors) according to which the model will train
y_vec=np.zeros((10962,5),dtype=float)
for i in range(10962):
    for j in range(5):
        p=review_data['Rating'][i]
        #print(p)
        #y_vec[i][0]=p
        #y_vec[i][1]=p
        #y_vec[i][2]=p
        #y_vec[i][3]=p
        #y_vec[i][4]=p
        y_vec[i][int(p)-1]=1

In [69]:
 X_train_cv, X_test_cv, y_train_cv, y_test_cv = model_selection.train_test_split(X_cv1, y_vec, test_size=0.33, random_state=42)


In [70]:
np.shape(y_train_cv)

(7344, 5)

In [71]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [72]:
model.fit(X_train_cv,y_train_cv,epochs=4,batch_size=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/4
7344/7344 [==============================] - 50s 7ms/step - loss: 0.8922 - acc: 0.6121
Epoch 2/4
7344/7344 [==============================] - 49s 7ms/step - loss: 0.6514 - acc: 0.7102
Epoch 3/4
7344/7344 [==============================] - 51s 7ms/step - loss: 0.3219 - acc: 0.8697
Epoch 4/4
7344/7344 [==============================] - 54s 7ms/step - loss: 0.1084 - acc: 0.9602


In [62]:
#input a sentence
sen=input("input sentence")
print(sen)
#now first we have to change this sentence to count vector
#since this sentence may contain some unknown words so we haveto take care of them also.

input sentence s


s


In [219]:
#count vectorizatin
inp_vec=np.zeros(shape=2642,dtype=int)
for i in range(2642):
    p=enstopvocab[i]
    for w in sen:
        if w==p:
            inp_vec[i]=inp_vec[i]+1
        else:
            inp_vec[2502]=inp_vec[2502]+1 # for "agyat" or 'unknown' category of words

In [220]:
print(sen)
print(inp_vec)

I hate it, there isn't any organization for the apps
[0 0 0 ... 0 0 0]


In [221]:
# multiplying with polarity
#for i in range(2642):
    #inp_vec[i]=inp_vec[i]*1

In [222]:
print(np.shape(inp_vec))
inp_vec_matrix=np.zeros(shape=(1,2642),dtype=float)
print(np.shape(inp_vec_matrix))
for j in range(2642):
        inp_vec_matrix[0][j]=inp_vec[j]

(2642,)
(1, 2642)


In [225]:
model.predict(inp_vec_matrix)

array([[0., 0., 0., 0., 1.]], dtype=float32)

In [224]:
for x in enstopvocab:
    if x=='hategjhgj':
        print("yea")